<a href="https://colab.research.google.com/github/kavy9/Numpy-and-pandas-prac/blob/main/MBTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/mbti_1.csv")

In [ ]:
df=df.iloc[:1000]

In [ ]:
import re
def post_cleaner(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    cleaned_text=[]
    for sentence in tqdm(data.posts):
        sentence=sentence.lower()
        
        #removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        #removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text

In [ ]:
df['posts']=post_cleaner(df)

NameError: ignored

In [ ]:
df.shape

(1000, 2)

In [ ]:
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer 
from keras_preprocessing.sequence import pad_sequences
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras
import transformers
from keras import backend as K
from nltk.stem import WordNetLemmatizer

In [ ]:
len(df['posts'].values[0])


4652

In [ ]:
posts=df['posts'].values
labels=df['type'].values

train_data,test_data = train_test_split(df,random_state=0,test_size=0.3)
len(train_data),len(test_data)

(6072, 2603)

In [ ]:
bert_model_name ='bert-base-uncased'
tokenizer=BertTokenizer.from_pretrained(bert_model_name,do_lower_case=True)
MAX_LEN = 64

def tokenize_sentences(sentences,tokenizer,max_seq_len=1800):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)
        
    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

train_input_ids = tokenize_sentences(train_data['posts'], tokenizer, MAX_LEN)
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_attention_masks = create_attention_masks(train_input_ids)

# test_input_ids = tokenize_sentences(test_data['posts'], tokenizer, MAX_LEN)
# test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
# test_attention_masks = create_attention_masks(test_input_ids)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


  0%|          | 0/6072 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def create_model(): 
    input_word_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32,
                                           name="input_word_ids")
    bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
    bert_outputs = bert_layer(input_word_ids)[0]
    pred = tf.keras.layers.Dense(16, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=pred)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00002), metrics=['accuracy', f1_m, precision_m, recall_m])
    return model

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.10.201.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.10.201.58:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFB

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 64)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 335141888 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                16400     
Total params: 335,158,288
Trainable params: 335,158,288
Non-trainable params: 0
_________________________________________________________________


In [ ]:
types = np.unique(df.type.values)

def get_type_index(string):
    return list(types).index(string)

In [ ]:
train_data['type_index'] = df['type'].apply(get_type_index)
train_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,type,posts,type_index
3162,INFP,'nvm|||Are you good at helping other people? S...,9
320,ENTP,"'Well, sorry, but I just think this is another...",3
6248,INFJ,'lol. http://youknowyoucare.com/wp-content/up...,8
2843,ENTP,'I thought personality cafe had a rate my pic ...,3
5281,INFP,'Same problem here. I cant get onto my origina...,9
...,...,...,...
4373,INFP,'Hey! It seems like you have a great foundatio...,9
7891,INFJ,"'Dear ISTJ mother, When I started my very fi...",8
4859,INTP,"'Oh ENTJs, how can you be scary and exciting a...",11
3264,ENFJ,"'Hi, ENTP, and welcome to the forum.:wink:|||F...",0


In [ ]:

one_hot_labels = tf.keras.utils.to_categorical(train_data.type_index.values, num_classes=16)

In [ ]:
BATCH_SIZE=8
NR_EPOCHS=16

In [ ]:
type(train_input_ids)

numpy.ndarray

In [ ]:
model.fit(np.array(train_input_ids), one_hot_labels, verbose = 1, epochs = NR_EPOCHS, batch_size = BATCH_SIZE,  callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5)])

Epoch 1/16


759/759 [==============================] - 227s 131ms/step - loss: 2.3314 - accuracy: 0.2047 - f1_m: 0.0012 - precision_m: 0.0012 - recall_m: 0.0012


Epoch 2/16
759/759 [==============================] - 99s 131ms/step - loss: 2.1691 - accuracy: 0.2887 - f1_m: 0.0478 - precision_m: 0.0478 - recall_m: 0.0478


Epoch 3/16
759/759 [==============================] - 99s 131ms/step - loss: 2.0572 - accuracy: 0.3282 - f1_m: 0.0893 - precision_m: 0.0893 - recall_m: 0.0893


Epoch 4/16
759/759 [==============================] - 100s 132ms/step - loss: 1.7709 - accuracy: 0.4274 - f1_m: 0.2024 - precision_m: 0.2024 - recall_m: 0.2024


...

Epoch 12/16
759/759 [==============================] - 99s 131ms/step - loss: 0.0747 - accuracy: 0.9794 - f1_m: 0.9766 - precision_m: 0.9766 - recall_m: 0.9766


Epoch 13/16
759/759 [==============================] - 99s 131ms/step - loss: 0.0666 - accuracy: 0.9792 - f1_m: 0.9781 - precision_m: 0.9781 - recall_m: 0.9781


Epoch 14/16
759/759 [==============================] - 99s 131ms/step - loss: 0.0809 - accuracy: 0.9755 - f1_m: 0.9728 - precision_m: 0.9728 - recall_m: 0.9728


Epoch 15/16
759/759 [==============================] - 99s 131ms/step - loss: 0.0620 - accuracy: 0.9806 - f1_m: 0.9781 - precision_m: 0.9781 - recall_m: 0.9781


Epoch 16/16
759/759 [==============================] - 99s 131ms/step - loss: 0.0525 - accuracy: 0.9837 - f1_m: 0.9814 - precision_m: 0.9814 - recall_m: 0.9814


In [ ]:
test_data['type_index'] = df['type'].apply(get_type_index)
test_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,type,posts,type_index
4587,ISFP,"'Dear ISFJ Mother, I wish you were less of a w...",13
2786,INFJ,"'To me, I think you guys may be over analyzing...",8
2813,ENFP,"'NIHM While NIHM has her INTJ husband, I've go...",1
3705,INTP,'I want 5 kids: - an astro/nuclear/theoretical...,11
5957,ISFP,'I have the same thing as well. I've noticed t...,13
...,...,...,...
2346,INTP,'YIKES! (When I wall posted you I hadn't read...,11
1814,ISFJ,I like eggs.|||I'm sure there is at least prim...,12
7695,INFJ,'I totally understand. I'm also strange and th...,8
3769,INTP,'Do you have money to have someone else do the...,11


In [ ]:
test_labels = tf.keras.utils.to_categorical(test_data.type_index.values, num_classes=16)

In [ ]:
test_input_ids = tokenize_sentences(test_data['posts'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

  0%|          | 0/2603 [00:00<?, ?it/s]

In [ ]:
model.evaluate(np.array(test_input_ids), test_labels)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 16) dtype=float32>]


82/82 [==============================] - 31s 142ms/step - loss: 5.7181 - accuracy: 0.2466 - f1_m: 0.2421 - precision_m: 0.2550 - recall_m: 0.2329


[5.718080043792725,
 0.2466384917497635,
 0.24206943809986115,
 0.2549542784690857,
 0.2328505963087082]

In [ ]:
cols = df['type'].unique()
cols = cols.tolist()

colnames = ['sentence']
colnames = colnames+cols

In [ ]:
df_predict = pd.DataFrame(columns = colnames)
sentence = "Time to debate on it. Strike at the weakest point and make others cry with facts"

df_predict.loc[0, 'sentence'] = sentence

In [ ]:
sentence_inputs = tokenize_sentences(df_predict['sentence'], tokenizer, MAX_LEN)
sentence_inputs = pad_sequences(sentence_inputs, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
prediction = model.predict(np.array(sentence_inputs))
df_predict.loc[0, cols] = prediction

df_predict

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 64) dtype=int64>]


,sentence,INFJ,ENTP,INTP,INTJ,ENTJ,ENFJ,INFP,ENFP,ISFP,ISTP,ISFJ,ISTJ,ESTP,ESFP,ESTJ,ESFJ
0,Time to debate on it. Strike at the weakest po...,0.334306,0.00104046,0.00144895,0.00281401,0.000106443,0.00032036,3.33145e-05,0.00504425,0.024735,0.447143,8.63573e-05,0.169229,0.000415382,0.00018588,0.00536268,0.00772843


In [ ]:
df_predict.iloc[:,2:].max(axis=1)

0    0.447143
dtype: float64